In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import datetime as dt
import numpy as np
import os

In [2]:
# Path to the XML file exported form Apple Health app
XML_DATA = "C:/PythonProjects/AppleHealth/input_files/apple_health_export/Export.xml"

In [3]:
# Parse XML file exported from Apple Health app
tree = ET.parse(XML_DATA)
root = tree.getroot()

In [4]:
attributes = []
for child in root:
    attributes.append(child.tag)

attributes = list(set(attributes))
print(attributes)

['Me', 'Record', 'Workout', 'ActivitySummary', 'ExportDate']


In [5]:
attributes.remove('ExportDate')
attributes.remove('Me')

print(attributes)

['Record', 'Workout', 'ActivitySummary']


### Workouts

In [6]:
level_1_list = []
level_2_list = []
level_3_list = []

a_list = list(root.iter('Workout'))
for a in range(len(a_list)):
    b_list = list(a_list[a])
    
    for b in range(len(b_list)):
        level_1_list.append(b_list[b].tag)
        c_list = list(b_list[b])

        for c in range(len(c_list)):
            level_2_list.append(c_list[c].tag)
            d_list = list(c_list[c])

            for d in range(len(d_list)):
                level_3_list.append(d_list[d].tag)

level_1_list = list(set(level_1_list))
level_2_list = list(set(level_2_list))
level_3_list = list(set(level_3_list))

In [7]:
### EMPTY LIST: no further levels to investigate
print(level_1_list)
print(level_2_list)
print(level_3_list)

concat_list = list(set(level_1_list + level_2_list + level_3_list))
print(concat_list)

['WorkoutEvent', 'WorkoutRoute', 'WorkoutStatistics', 'MetadataEntry']
['MetadataEntry', 'FileReference']
[]
['WorkoutEvent', 'MetadataEntry', 'FileReference', 'WorkoutStatistics', 'WorkoutRoute']


In [8]:
index = 0
workouts = root.findall('.//Workout')
total_workouts = len(workouts)

for wo in workouts:
    index = index + 1
    wo.attrib['workout_pk'] = str(index)

    for wo_2 in level_1_list:
        workouts_2 = wo.findall(f'.//{wo_2}')
        for x in workouts_2:
            x.attrib['workout_pk'] = str(index)

            for wo_3 in level_2_list:
                workouts_3 = x.findall(f'.//{wo_3}')
                for y in workouts_3:
                    y.attrib['workout_pk'] = str(index)

tree.write('./output_files/Indexed_XML.xml')

In [9]:
# add Workout to list
concat_list.append('Workout')

In [10]:
tree = ET.parse('./output_files/Indexed_XML.xml')
root = tree.getroot()

In [11]:
df_list = []
for elem in concat_list:
    workouts = root.findall(f'.//{elem}')
    globals()[f"{elem}_list"] = []
    df_list.append(str(f"{elem}_list"))
    for wo in workouts:
        globals()[f"{elem}_list"].append(wo.attrib)

In [12]:
df_list

['WorkoutEvent_list',
 'MetadataEntry_list',
 'FileReference_list',
 'WorkoutStatistics_list',
 'WorkoutRoute_list',
 'Workout_list']

In [13]:
for elem in df_list:
    object = globals()[elem]
    df_name = f"df_{elem}"
    print(f"{df_name}:")
    globals()[df_name] = pd.DataFrame(object)
    globals()[df_name]['workout_pk'] = pd.to_numeric(globals()[df_name]['workout_pk'])
    display(pd.DataFrame(object).head(2))

df_WorkoutEvent_list:


,type,date,duration,durationUnit,workout_pk
0,HKWorkoutEventTypeSegment,2022-03-29 15:56:57 +0100,13.28115280667941,min,72
1,HKWorkoutEventTypeSegment,2022-03-29 15:56:57 +0100,25.37857104341189,min,72


df_MetadataEntry_list:


,key,value,workout_pk
0,HKWasUserEntered,1,NaN
1,HKWasUserEntered,1,NaN


df_FileReference_list:


...

,type,startDate,endDate,sum,unit,workout_pk,average,minimum,maximum
0,HKQuantityTypeIdentifierActiveEnergyBurned,2021-12-02 18:38:33 +0100,2021-12-02 18:58:33 +0100,254,kcal,1,NaN,NaN,NaN
1,HKQuantityTypeIdentifierDistanceCycling,2021-12-02 18:38:33 +0100,2021-12-02 18:58:33 +0100,8.09,km,1,NaN,NaN,NaN


df_WorkoutRoute_list:


,sourceName,sourceVersion,creationDate,startDate,endDate,workout_pk
0,Apple Watch von Lucas,8.1,2022-03-29 16:39:48 +0100,2022-03-29 15:57:11 +0100,2022-03-29 16:39:47 +0100,72
1,Apple Watch von Lucas,8.1,2022-04-03 10:15:12 +0100,2022-04-03 09:48:45 +0100,2022-04-03 10:15:10 +0100,81


df_Workout_list:


,workoutActivityType,duration,durationUnit,sourceName,sourceVersion,creationDate,startDate,endDate,workout_pk,device
0,HKWorkoutActivityTypeCycling,20,min,Peloton,349778,2022-03-28 19:53:49 +0100,2021-12-02 18:38:33 +0100,2021-12-02 18:58:33 +0100,1,NaN
1,HKWorkoutActivityTypeCycling,20,min,Peloton,349778,2022-03-28 19:53:50 +0100,2021-12-03 06:17:51 +0100,2021-12-03 06:37:51 +0100,2,NaN


#### Transform dfs

##### WorkoutStatistics

In [14]:
df_workoutstatistics_melt = pd.melt(df_WorkoutStatistics_list, 
                                    id_vars=['workout_pk', 
                                             'type', 
                                             'unit'], 
                                    value_vars=['sum', 
                                                'average', 
                                                'minimum', 
                                                'maximum'], 
                                    var_name='metric',
                                    value_name='value').dropna()

# shorter observation names
df_workoutstatistics_melt['type'] = df_workoutstatistics_melt['type'].str.replace('HKQuantityTypeIdentifier', '')
df_workoutstatistics_melt['type'] = df_workoutstatistics_melt['type'].str.replace('HKCategoryTypeIdentifier', '')

list(df_workoutstatistics_melt['metric'].unique())

['sum', 'average', 'minimum', 'maximum']

In [15]:
mapMetrics = {'sum': 'Sum', 
              'average': 'Avg',
              'minimum': 'Min',
              'maximum': 'Max'}

df_workoutstatistics_melt = df_workoutstatistics_melt.replace({'metric': mapMetrics})
df_workoutstatistics_melt['unit'] = df_workoutstatistics_melt['unit'].str.lower()

df_workoutstatistics_melt['type'] = df_workoutstatistics_melt['unit'] + "_" + df_workoutstatistics_melt['type'] + df_workoutstatistics_melt['metric']
df_workoutstatistics_melt_drop = df_workoutstatistics_melt.drop(['unit', 'metric'], axis = 1)

In [16]:
df_workoutstatistics_final = df_workoutstatistics_melt_drop.pivot(index='workout_pk', columns='type', values='value')

##### Workouts

##### Prepare DimWorkout

In [17]:
list(df_Workout_list.columns)

['workoutActivityType',
 'duration',
 'durationUnit',
 'sourceName',
 'sourceVersion',
 'creationDate',
 'startDate',
 'endDate',
 'workout_pk',
 'device']

In [18]:
df_Workouts = df_Workout_list.drop(['device', 'sourceVersion'], axis = 1)

df_Workouts['workout_pk'] = pd.to_numeric(df_Workouts['workout_pk'])
df_Workouts['Date'] = pd.to_datetime(df_Workouts['startDate']).dt.strftime('%Y-%m-%d')
df_Workouts['startDate'] = pd.to_datetime(df_Workouts['startDate'])
df_Workouts['endDate'] = pd.to_datetime(df_Workouts['endDate'])
df_Workouts['workoutActivityType'] = df_Workouts['workoutActivityType'].str.replace('HKWorkoutActivityType', '')

In [19]:
DimWorkouts_prep = pd.merge(df_Workouts, df_workoutstatistics_final, on = 'workout_pk', how = 'left')
DimWorkouts_prep.shape

(600, 35)

In [20]:
DimWorkouts = pd.merge(DimWorkouts_prep, df_FileReference_list, on = 'workout_pk', how='left')

### Records

In [21]:
records_list = [x.attrib for x in root.iter('Record')]
df_records = pd.DataFrame(records_list)

df_records = df_records.drop(['sourceName','sourceVersion', 'device'], axis=1)

# parse date columns
df_records['Date'] = pd.to_datetime(df_records['startDate']).dt.strftime('%Y-%m-%d')
df_records['startTime'] = pd.to_datetime(df_records['startDate']).dt.strftime('%H:%M:%S')
df_records['endTime'] = pd.to_datetime(df_records['endDate']).dt.strftime('%H:%M:%S')

# value is numeric, NaN if fails
df_records['value'] = pd.to_numeric(df_records['value'], errors='coerce')

# shorter observation names
df_records['type'] = df_records['type'].str.replace('HKQuantityTypeIdentifier', '')
df_records['type'] = df_records['type'].str.replace('HKCategoryTypeIdentifier', '')

In [22]:

record_types = list(df_records.type.unique())
print(record_types)

['Height', 'BodyMass', 'HeartRate', 'OxygenSaturation', 'RespiratoryRate', 'StepCount', 'DistanceWalkingRunning', 'BasalEnergyBurned', 'ActiveEnergyBurned', 'FlightsClimbed', 'AppleExerciseTime', 'DistanceCycling', 'DistanceSwimming', 'SwimmingStrokeCount', 'RestingHeartRate', 'VO2Max', 'WalkingHeartRateAverage', 'DistanceDownhillSnowSports', 'EnvironmentalAudioExposure', 'HeadphoneAudioExposure', 'WalkingDoubleSupportPercentage', 'SixMinuteWalkTestDistance', 'AppleStandTime', 'WalkingSpeed', 'WalkingStepLength', 'WalkingAsymmetryPercentage', 'StairAscentSpeed', 'StairDescentSpeed', 'AtrialFibrillationBurden', 'AppleWalkingSteadiness', 'NumberOfAlcoholicBeverages', 'RunningStrideLength', 'RunningVerticalOscillation', 'RunningGroundContactTime', 'HeartRateRecoveryOneMinute', 'RunningPower', 'RunningSpeed', 'TimeInDaylight', 'PhysicalEffort', 'SleepAnalysis', 'AppleStandHour', 'LowHeartRateEvent', 'HeartRateVariabilitySDNN']


In [23]:

personal_records = ['Height', 
                    'BodyMass',
                    'RestingHeartRate',
                    'WalkingHeartRateAverage',
                    'AppleWalkingSteadiness',
                    'SixMinuteWalkTestDistance']

basis_records = ['HeartRate', 
                 'OxygenSaturation', 
                 'RespiratoryRate', 
                 'StepCount', 
                 'FlightsClimbed', 
                 'AppleExerciseTime',
                 'VO2Max',
                 'EnvironmentalAudioExposure', 
                 'HeadphoneAudioExposure',
                 'WalkingDoubleSupportPercentage', 
                 'AppleStandTime', 
                 'WalkingSpeed', 
                 'WalkingStepLength', 
                 'WalkingAsymmetryPercentage',
                 'StairAscentSpeed', 
                 'StairDescentSpeed',
                 'SleepAnalysis', 
                 'AppleStandHour', 
                 'LowHeartRateEvent', 
                 'HeartRateVariabilitySDNN',
                 'HeartRateRecoveryOneMinute']

workout_records = ['DistanceWalkingRunning', 
                   'BasalEnergyBurned', 
                   'ActiveEnergyBurned', 
                   'DistanceCycling', 
                   'DistanceSwimming', 
                   'SwimmingStrokeCount',
                   'DistanceDownhillSnowSports',
                   'RunningStrideLength', 
                   'RunningVerticalOscillation', 
                   'RunningGroundContactTime', 
                   'RunningPower',
                   'RunningSpeed']


##### Personal Records

In [24]:
# dictionary of DataFrames for filtered 'record_data'
personal_dict = {}

# create new DataFrame for every interested data
for type in personal_records:
   print(type)
   personal_dict[type] = df_records[df_records['type'] == type].rename(columns={"value": type}).sort_values(by='Date')

   # create from/to-date columns --> drop irrelevant columns
   personal_dict[type]['fromDate'] = personal_dict[type]['Date']
   personal_dict[type]['toDate'] = personal_dict[type]['Date'].shift(-1)
   personal_dict[type] = personal_dict[type].drop(['creationDate', 'startDate', 'endDate', 'Date', 'startTime', 'endTime'], axis = 1)

   #display(personal_dict[type].tail(5))

Height
BodyMass
RestingHeartRate
WalkingHeartRateAverage
AppleWalkingSteadiness
SixMinuteWalkTestDistance


In [25]:
personal_record_df = pd.DataFrame()

for type in personal_records:
    frames = [personal_record_df, personal_dict[type]]
    personal_record_df = pd.concat(frames)

In [26]:
FactPersonalRecords = pd.melt(personal_record_df,
                      id_vars=['fromDate',
                               'toDate',
                               'unit'], 
                     value_vars=personal_records, 
                     var_name='type',
                     value_name='value')

FactPersonalRecords = FactPersonalRecords[FactPersonalRecords['value'].notna()]
FactPersonalRecords = FactPersonalRecords[['type', 'value', 'unit', 'fromDate', 'toDate']]

In [27]:
FactPersonalRecords

,type,value,unit,fromDate,toDate
0,Height,181.0,cm,2022-03-28,2022-04-30
1,Height,181.0,cm,2022-04-30,None
1336,BodyMass,80.9,kg,2017-06-10,2017-06-27
1337,BodyMass,80.9,kg,2017-06-27,2018-02-07
1338,BodyMass,82.7,kg,2018-02-07,2018-03-11
...,...,...,...,...,...
7999,SixMinuteWalkTestDistance,500.0,m,2023-10-02,2023-10-09
8000,SixMinuteWalkTestDistance,500.0,m,2023-10-09,2023-10-17
8001,SixMinuteWalkTestDistance,500.0,m,2023-10-17,2023-10-24
8002,SixMinuteWalkTestDistance,500.0,m,2023-10-24,2023-10-31


##### Basis Records

In [28]:
# dictionary of DataFrames for filtered 'record_data'
basis_dict = {}

# create new DataFrame for every interested data
for type in basis_records:
   print(type)
   basis_dict[type] = df_records[df_records['type'] == type].rename(columns={"value": type}).sort_values(by='Date')

   #display(basis_dict[type].tail(3))

HeartRate


OxygenSaturation
RespiratoryRate
StepCount
FlightsClimbed
AppleExerciseTime
VO2Max
EnvironmentalAudioExposure
HeadphoneAudioExposure
WalkingDoubleSupportPercentage
AppleStandTime
WalkingSpeed
WalkingStepLength
WalkingAsymmetryPercentage
StairAscentSpeed
StairDescentSpeed
SleepAnalysis
AppleStandHour
LowHeartRateEvent
HeartRateVariabilitySDNN
HeartRateRecoveryOneMinute


In [29]:
basis_record_df = pd.DataFrame()

for type in basis_records:
    frames = [basis_record_df, basis_dict[type]]
    basis_record_df = pd.concat(frames)

basis_record_df = basis_record_df.drop(['creationDate', 'startTime', 'endTime'], axis = 1)

In [30]:
FactBasisRecords = pd.melt(basis_record_df,
                      id_vars=['unit',
                               'startDate',
                               'endDate',
                               'Date'], 
                     value_vars=basis_records, 
                     var_name='type',
                     value_name='value')

FactBasisRecords = FactBasisRecords[FactBasisRecords['value'].notna()]
FactBasisRecords = FactBasisRecords[['type', 'value', 'unit', 'Date', 'startDate', 'endDate']]

In [31]:
FactBasisRecords['startDate'] = pd.to_datetime(FactBasisRecords['startDate'])
FactBasisRecords['endDate'] = pd.to_datetime(FactBasisRecords['endDate'])
FactBasisRecords['Date'] = pd.to_datetime(FactBasisRecords['Date'])


# Create dataframe that lists workout Starts/Ends
wo_start_end = DimWorkouts[['workout_pk', 'Date', 'startDate', 'endDate']].drop_duplicates()
wo_start_end['startDate'] = pd.to_datetime(wo_start_end['startDate'])
wo_start_end['endDate'] = pd.to_datetime(wo_start_end['endDate'])
wo_start_end['Date'] = pd.to_datetime(wo_start_end['Date'])

FactBasisRecordsMerged = pd.merge(FactBasisRecords, wo_start_end, on = 'Date', how='left', suffixes=('_records', '_wo'))
FactBasisRecordsMerged['workout_pk'] = FactBasisRecordsMerged['workout_pk'].astype('Int64')

FactBasisRecordsMerged = FactBasisRecordsMerged[
    ((FactBasisRecordsMerged['startDate_records'] >= FactBasisRecordsMerged['startDate_wo']) & 
    (FactBasisRecordsMerged['startDate_records'] <= FactBasisRecordsMerged['endDate_wo'])) |
    (FactBasisRecordsMerged['startDate_wo'].isnull())
    ]

FactBasisRecords = FactBasisRecordsMerged.drop(['startDate_wo', 'endDate_wo'], axis = 1)

##### Workout Records

In [32]:
# dictionary of DataFrames for filtered 'record_data'
workout_dict = {}

# create new DataFrame for every interested data
for type in workout_records:
   print(type)
   workout_dict[type] = df_records[df_records['type'] == type].rename(columns={"value": type}).sort_values(by='Date')

   #display(workout_dict[type].tail(3))

DistanceWalkingRunning
BasalEnergyBurned
ActiveEnergyBurned
DistanceCycling
DistanceSwimming
SwimmingStrokeCount
DistanceDownhillSnowSports
RunningStrideLength
RunningVerticalOscillation
RunningGroundContactTime
RunningPower
RunningSpeed


In [33]:
workout_record_df = pd.DataFrame()

for type in workout_records:
    frames = [workout_record_df, workout_dict[type]]
    workout_record_df = pd.concat(frames)

workout_record_df = workout_record_df.drop(['creationDate', 'startTime', 'endTime'], axis = 1)

In [34]:
FactWorkoutRecords = pd.melt(workout_record_df,
                      id_vars=['unit',
                               'startDate',
                               'endDate',
                               'Date'], 
                     value_vars=workout_records, 
                     var_name='type',
                     value_name='value')

FactWorkoutRecords = FactWorkoutRecords[FactWorkoutRecords['value'].notna()]
FactWorkoutRecords = FactWorkoutRecords[['type', 'value', 'unit', 'Date', 'startDate', 'endDate']]

In [35]:
FactWorkoutRecords['startDate'] = pd.to_datetime(FactWorkoutRecords['startDate'])
FactWorkoutRecords['endDate'] = pd.to_datetime(FactWorkoutRecords['endDate'])
FactWorkoutRecords['Date'] = pd.to_datetime(FactWorkoutRecords['Date'])

# Create dataframe that lists workout Starts/Ends
wo_start_end = DimWorkouts[['workout_pk', 'Date', 'startDate', 'endDate']].drop_duplicates()
wo_start_end['startDate'] = pd.to_datetime(wo_start_end['startDate'])
wo_start_end['endDate'] = pd.to_datetime(wo_start_end['endDate'])
wo_start_end['Date'] = pd.to_datetime(wo_start_end['Date'])

FactWorkoutRecordsMerged = pd.merge(FactWorkoutRecords, wo_start_end, on = 'Date', how='left', suffixes=('_records', '_wo'))
FactWorkoutRecordsMerged['workout_pk'] = FactWorkoutRecordsMerged['workout_pk'].astype('Int64')

FactWorkoutRecordsMerged = FactWorkoutRecordsMerged[
    (FactWorkoutRecordsMerged['startDate_records'] >= FactWorkoutRecordsMerged['startDate_wo']) & 
    (FactWorkoutRecordsMerged['startDate_records'] <= FactWorkoutRecordsMerged['endDate_wo'])
    ]

FactWorkoutRecords = FactWorkoutRecordsMerged

In [36]:
#################################################################################################################

### Workout Routes

In [37]:
# Path to the XML file exported form Apple Health app
XML_routes = "C:/PythonProjects/AppleHealth/input_files/apple_health_export/workout-routes"

In [38]:
example_route = list(os.listdir(XML_routes))[0]

tree = ET.parse(f'{XML_routes}/{example_route}')
root = tree.getroot()

In [39]:
attributes = []
for child in root:
    attributes.append(child.tag)

attributes = list(set(attributes))
print(attributes)

['{http://www.topografix.com/GPX/1/1}trk', '{http://www.topografix.com/GPX/1/1}metadata']


In [40]:
level_1_list = []
level_2_list = []
level_3_list = []
level_4_list = []
level_5_list = []

a_list = list(root.iter('{http://www.topografix.com/GPX/1/1}trk'))
for a in range(len(a_list)):
    b_list = list(a_list[a])
    
    for b in range(len(b_list)):
        level_1_list.append(b_list[b].tag)
        c_list = list(b_list[b])

        for c in range(len(c_list)):
            level_2_list.append(c_list[c].tag)
            d_list = list(c_list[c])

            for d in range(len(d_list)):
                level_3_list.append(d_list[d].tag)
                e_list = list(d_list[d])

                for e in range(len(e_list)):
                    level_4_list.append(e_list[e].tag)
                    f_list = list(e_list[e])

                    for f in range(len(f_list)):
                        level_5_list.append(f_list[f].tag)

level_1_list = list(set(level_1_list))
level_2_list = list(set(level_2_list))
level_3_list = list(set(level_3_list))
level_4_list = list(set(level_4_list))
level_5_list = list(set(level_5_list))

In [41]:
### EMPTY LIST: no further levels to investigate
print(level_1_list)
print(level_2_list)
print(level_3_list)
print(level_4_list)
print(level_5_list)

concat_list = list(set(level_1_list + level_2_list + level_3_list + level_4_list))

['{http://www.topografix.com/GPX/1/1}trkseg', '{http://www.topografix.com/GPX/1/1}name']
['{http://www.topografix.com/GPX/1/1}trkpt']
['{http://www.topografix.com/GPX/1/1}extensions', '{http://www.topografix.com/GPX/1/1}ele', '{http://www.topografix.com/GPX/1/1}time']
['{http://www.topografix.com/GPX/1/1}vAcc', '{http://www.topografix.com/GPX/1/1}hAcc', '{http://www.topografix.com/GPX/1/1}speed', '{http://www.topografix.com/GPX/1/1}course']
[]


In [42]:
route_metrics = ['trkpt', 'ele', 'time', 'course', 'speed', 'hAcc', 'vAcc']

In [43]:
combined_routes_df = pd.DataFrame()

for file in list(os.listdir(XML_routes)):
    route_df = pd.DataFrame()
    tree = ET.parse(f'{XML_routes}/{file}')

    for metric in route_metrics:
        # create metric lists
        globals()[f"list_{metric}"] = []
        source = "{http://www.topografix.com/GPX/1/1}"+str(metric)

        # loop elements of metric
        for elem in tree.findall(f".//{source}"):

            # add elements to metric lists (trkpt's have different format)
            if elem.tag == '{http://www.topografix.com/GPX/1/1}trkpt':
                globals()[f"list_{metric}"].append(elem.attrib)
            else:
                globals()[f"list_{metric}"].append(elem.text)

        # check whether column is time (TIME HAS ALWAYS N+1 ENTRIES --> ignore first entry) 
        if metric == 'time':
            route_df[metric] = globals()[f"list_{metric}"][1:]
        else:
            route_df[metric] = globals()[f"list_{metric}"]

    # add route name as column
    route_df['route_name'] = file
    # union "route" dataframe to overall "combined_routes" dataframe
    combined_routes_df = pd.concat([combined_routes_df, route_df])    

print(f"{len(list(combined_routes_df['route_name'].unique()))} routes added to dataframe!")

347 routes added to dataframe!


In [44]:
DimWorkouts['route_name'] = DimWorkouts['path'].str.split(pat = '/')
DimWorkouts['route_name'] = DimWorkouts['route_name'].str[-1]
DimRoutes = pd.merge(combined_routes_df, DimWorkouts[['route_name', 'workout_pk']], on = 'route_name', how = 'left')

### Save Tables

In [45]:
FactBasisRecords.to_parquet('./output_files/frontend_files/FactBasisRecords.parquet.gzip', compression='gzip')
FactWorkoutRecords.to_parquet('./output_files/frontend_files/FactWorkoutRecords.parquet.gzip', compression='gzip')
FactPersonalRecords.to_parquet('./output_files/frontend_files/FactPersonalRecords.parquet.gzip', compression='gzip')
DimWorkouts.to_parquet('./output_files/frontend_files/DimWorkouts.parquet.gzip', compression='gzip')
DimRoutes.to_parquet('./output_files/frontend_files/DimRoutes.parquet.gzip', compression='gzip')